## Correct Annotation files
Validate and correct annotation files

Input annotation file is roLabelImg file

In [293]:
import copy
import os
import xml.etree.ElementTree as ET


class Xml:
    def __init__(self, tree):
        """Clone given xml file and empty objects
        
        Args:
            tree(xml.etree.ElementTree.ElementTree): XML tree object
        """
        self.tree = copy.deepcopy(tree)
        self.root = self.tree.getroot()
        for obj in self.root.findall('object'):
            self.root.remove(obj)
        
    def add(self, obj):
        """Add object into the xml file
        
        Args:
            obj(Object): Object to be added
        """
        self.root.append(obj)
        
    def save(self, path):
        self.tree.write(path)
        

def get_shape(obj_box):
    """Get a shape of a box
    
    Args:
        obj_box(Element): box object
        
    Returns:
        width(int or float): Width of a box
        height(int or float): Height of a box
    """
    if obj_box.tag=='bndbox':
        box = [float(value.text) for value in obj_box]  # [xmin, ymin, xmax, ymax] Integer
        width = abs(box[2] - box[0])
        height = abs(box[3] - box[1])
    elif obj_box.tag == 'robndbox':        
        box = [float(value.text) for value in obj_box]  # [x1, y1, ..., x4, y4, cx, cy, w, h, angle] Float        
        width = box[10]
        height = box[11]
    else:
        raise Exception('Wrong box type: ', obj_box.tag)
        
    return width, height

def convert_box_type(obj):
    """Convert box type between bndbox and robndbox
    
    If a given box type is bndbox, return robndbox.
    If a given box type is robndbox, return bndbox.
    
    Args:
        obj(Element): object having box element
        
    Returns:
        obj(Element): converted object
    """
    def reverse_rotation(x, y, cx, cy, angle):
        """Restore original x, y coordinates to which before it was rotated
        
        Args:
            x(float): x coordinate
            y(float): x coordinate
            cx(float): cx coordinate
            cy(float): cy coordinate
            angle(float): angle
        
        Returns:
            rx(float): restored x coordinate
            ry(float): restored y coordinate
        """
        tx = x - cx
        ty = y - cy

        rx = tx * math.cos(angle) + ty * math.sin(angle) + cx
        ry = ty * math.cos(angle) - tx * math.sin(angle) + cy

        return rx, ry
    
    if obj.find('bndbox') is not None:
        box = [round(float(value.text)) for value in obj.find('bndbox')]  # [xmin, ymin, xmax, ymax]
        obj.remove(obj.find('bndbox'))
        robndbox = {'x1': str(box[0]), 'y1': str(box[1]),
                   'x2': str(box[2]), 'y2': str(box[1]),
                   'x3': str(box[2]), 'y3': str(box[3]),
                   'x4': str(box[0]), 'y4': str(box[3]),
                   'cx': str((box[2] - box[0]) / 2),
                   'cy': str((box[3] - box[1]) / 2),
                   'w': str(box[2] - box[0]),
                   'h': str(box[3] - box[1]),
                   'angle': '0.0'}
        e_robndbox = ET.Element('robndbox')
        for key, value in robndbox.items():
            e_sub = ET.SubElement(e_robndbox, key)
            e_sub.text = value
        obj.append(e_robndbox)
        obj.find('type').text = 'robndbox'
    elif obj.find('robndbox') is not None:
        box = [float(value.text) for value in obj.find('robndbox')]  # [x1, y1, ..., x4, y4, cx, cy, w, h, angle]        
        obj.remove(obj.find('robndbox'))
        
        xmin, ymin = reverse_rotation(box[0], box[1], box[8], box[9], box[12])
        xmax, ymax = reverse_rotation(box[4], box[5], box[8], box[9], box[12])        
        bndbox = {'xmin': str(round(xmin)),
                 'ymin': str(round(ymin)),
                 'xmax': str(round(xmax)),
                 'ymax': str(round(ymax))}
        e_bndbox = ET.Element('bndbox')
        for key, value in bndbox.items():
            e_sub = ET.SubElement(e_bndbox, key)
            e_sub.text = value
        obj.append(e_bndbox)
        obj.find('type').text = 'bndbox'
    else:
        raise Exception('Box type is not supported: ', obj)
        
    return obj

def valdiate_obj(obj):
    """Validate object
    
    Validate if an object has all attributes
    Attributes : [type, name, pose, truncated, difficult, robndbox or bndbox]
    
    Sometimes, robndbox's [x, y] coordinates don't exist but [cx, cy, w, h, angle] do.
    In that case, calculate it using its cx, cy, w, h, angle.
    If a robndbox is just empty, return False.
    
    Args:
        obj(Element): object
    Returns:
        True(bool): If it is validate object otherwise return false    
    """
    def restore_robndbox(robndbox):
        """Restore x,y coordinates of a robndbox
        
        Args:
            robndbox(Element): robndbox element
        
        Returns:
            restored_robndbox(Element): restored robndbox element
        """
        cx = float(robndbox.find('cx').text)
        cy = float(robndbox.find('cy').text)
        w = float(robndbox.find('w').text)
        h = float(robndbox.find('h').text)
        angle = float(robndbox.find('angle').text)
        
        t_xs = [cx - w/2, cx + w/2, cx + w/2, cx - w/2]
        t_ys = [cy - h/2, cy - h/2, cy + h/2, cy + h/2]

        xs = []
        ys = []
        for t_x, t_y in zip(t_xs, t_ys):
            x, y = cvt_angle(t_x, t_y, cx, cy, angle)
            xs.append(x)
            ys.append(y)            
            
        new_robndbox = {'x1': str(x1), 'y1': str(y1),
                        'x2': str(x2), 'y2': str(y2),
                        'x3': str(x3), 'y3': str(y3),
                        'x4': str(x4), 'y4': str(y4),
                        'cx': str(cx), 'cy': str(cy),
                        'w': str(w), 'h': str(h),
                        'angle': str(angle)}
        
        restored_robndbox = ET.Element('robndbox')        
        for key, value in new_robndbox.items():
            e_sub = ET.SubElement(restored_robndbox, key)
            e_sub.text = value
    
        return restored_robndbox
    
    attributes = [attr.tag for attr in obj]
    
    if 'type' not in attributes:
        print('type false')
        return False
    
    if 'name' not in attributes:
        print('name false')
        return False
    
    if 'bndbox' in attributes:
        attr_box = [attr.tag for attr in obj.find('bndbox')]
        val_attr = ['xmin', 'ymin', 'xmax', 'ymax']
        for attr in val_attr:
            if attr not in attr_box:
                print('bndbox false: ', attr_box)
                return False
    elif 'robndbox' in attributes:
        attr_box = [attr.tag for attr in obj.find('robndbox')]
        val_attr = ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'cx', 'cy', 'w', 'h', 'angle']
        restore_attr = ['cx', 'cy', 'w', 'h', 'angle']
        
        # If any attribute is missing, check if it is restorable
        for v_attr in val_attr:
            if v_attr not in attr_box:
                for r_attr in restore_attr:
                    if r_attr not in attr_box:
                        print('restoration failed')  # temp
                        return False
                break
        restored_robndbox = restore_robndbox(obj.find('robndbox'))
        obj.remove(obj.find('robndbox'))
        obj.append(restored_robndbox)
    
    return True

In [295]:
# User Setting
sensor = 'S1'  # S1: Sentinel-1, K5: KOMPSAT-5
home = os.path.join('work', '230920')

# Automatic
cor_home = os.path.join(os.path.dirname(home), os.path.basename(home) + '_correction')
os.makedirs(cor_home, exist_ok=True)

err_file_path = os.path.join(os.path.dirname(home), 'Error_list.txt')
cor_file_path = os.path.join(os.path.dirname(home), 'Corrected_list.txt')
empty_file_path = os.path.join(os.path.dirname(home), 'Empty_list.txt')

s1_classes = ('ship', 'wind farm')
k5_classes = ('ship', 'oil tank', 'wind farm', 'floating oil tank', 'fixed oil tank')
box_classes = {'ship': 'robndbox', 'oil tank': 'bndbox', 'wind farm': 'bndbox', 'floating oil tank': 'bndbox', 'fixed oil tank': 'bndbox'}

sensor_classes = {'K5': k5_classes, 'S1': s1_classes}
classes = sensor_classes[sensor]

err_ann = []
cor_ann = []
empty_ann = []

err_info = ''
cor_info = ''

print(f'Validating and Correcting {len(os.listdir(home))} files')
for ann_name in os.listdir(home):
    ann_path = os.path.join(home, ann_name)

    tree = ET.parse(ann_path)
    new_xml = Xml(tree)
    
    root = tree.getroot()
    objs = root.findall('object')
    
    if not objs:
        empty_ann.append(ann_name)
        
    for obj in objs:
        if not valdiate_obj(obj):
            err_info = f'ann: {ann_name}\ncause: Wrong object'
            break
        obj_cls = obj.find('name').text
        if obj_cls in classes:
            if box_classes[obj_cls] != obj.find('type').text:
                obj = convert_box_type(obj)
                cor_info = ann_name
                
            obj_box = obj.find(box_classes[obj_cls])
            
            width, height = get_shape(obj_box)            
            if width * height < 4:
                err_info = f'ann: {ann_name}\ncause: Wrong box size'
                break
            else:
                new_xml.add(obj)
        else:
            err_info = f'ann: {ann_name}\ncause: Wrong class ({obj_cls})'
            break
            
    if err_info:
        err_ann.append(err_info)
        err_info = ''
        continue
        
    if cor_info:
        cor_ann.append(ann_name)
        
    cor_info = ''
    new_xml.save(os.path.join(cor_home, ann_name))
    
if err_ann:
    with open(err_file_path, 'w') as f:
        for file in err_ann:
            f.write(file + '\n')
    
if cor_ann:
    with open(cor_file_path, 'w') as f:
        for file in cor_ann:
            f.write(file + '\n')
            
if empty_ann:
    with open(empty_file_path, 'w') as f:
        for file in empty_ann:
            f.write(file + '\n')
            
print('Errors: ', len(err_ann))
print('Corrected: ', len(cor_ann))
print('Empty: ', len(empty_ann))
print('Done')

Validating and Correcting 3870 files
Errors:  0
Corrected:  14
Empty:  354
Done


## Rotation test code

In [296]:
import math

def cvt_angle(x, y, cx, cy, angle):
    tx = x - cx
    ty = y - cy
    
    rx = tx * math.cos(angle) - ty * math.sin(angle) + cx
    ry = ty * math.cos(angle) + tx * math.sin(angle) + cy
    
    return rx, ry

def rcvt_angle(x, y, cx, cy, angle):
    tx = x - cx
    ty = y - cy
    
    rx = tx * math.cos(angle) + ty * math.sin(angle) + cx
    ry = ty * math.cos(angle) - tx * math.sin(angle) + cy
    
    return rx, ry

x = 10.5
y = 50.0
cx = 30.5
cy = 30.0
#angle = 90 * math.pi / 180
angle = 0.1  # radian

rx, ry = cvt_angle(x, y, cx, cy, angle)
x2, y2 = rcvt_angle(rx, ry, cx, cy, angle)
print('x 오차: ', x2 - x)
print('y 오차: ', y2 - y)

x 오차:  0.0
y 오차:  7.105427357601002e-15
